In [1]:
from main import *
import numpy as np
import sys

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
def genpairs(n, m, N):
    L=set()
    i, j = np.random.randint(0, n), np.random.randint(0, m)
    L.add((i,j))
    while len(L) < N:
        while (i, j) in L:
            i, j = np.random.randint(0, n), np.random.randint(0,m)
        L.add((i,j))
    return L
def genpairsnotinL(n,m, N, LL):
    L=set()
    i, j = np.random.randint(0, n), np.random.randint(0, m)
    if (i,j) not in LL:
        L.add((i,j))
    while len(L) < N:
        while (i, j) in L or (i,j) in LL:
            i, j = np.random.randint(0, n), np.random.randint(0,m)
        L.add((i,j))
    return L

def IsAutonomous(input_matrix, output_matrix, aut_species, aut_reactions):
    
    autonomous_species=True
    autonomous_reactions=True
    #n, m =input_matrix.shape
    for s in aut_species:
        if sum(input_matrix[s,r] for r in aut_reactions)<0.5 or sum(output_matrix[s,r] for r in aut_reactions)<0.5:
            autonomous_species=False
    for r in aut_reactions:
        if sum(input_matrix[s,r] for s in aut_species)<0.5 or sum(output_matrix[s,r] for s in aut_species)<0.5:
            autonomous_reactions=False

    return autonomous_species, autonomous_reactions


## Data

In [3]:
# NN=[10, 20, 30, 40, 50]
# MM=[10, 20, 30, 40, 50]
# UB=100
# D=[3,4,5]
# for n in NN:
#     for m in MM:
#         for d in D:
#             for rep in [1,2,3,4,5]:
#                 matrix = np.random.randint(0,UB, size=(n,m))
#                 pairs1=genpairs(n, m, n*m/d)
#                 pairs2=genpairsnotinL(n, m, n*m/d, pairs1)

#                 output_matrix = np.zeros((n,m))
#                 input_matrix = np.zeros((n,m))
#                 for i in range(n):
#                     for j in range(m):
#                         if (i,j) in pairs1:
#                             input_matrix[i,j]=matrix[i,j]
#                         elif (i,j) in pairs2:
#                             output_matrix[i,j]=matrix[i,j]

#                 np.savetxt("instances/left_n%dm%dd%d_%d.txt"%(n,m,d,rep), input_matrix, fmt="%d")
#                 np.savetxt("instances/right_n%dm%dd%d_%d.txt"%(n,m,d,rep), output_matrix, fmt="%d")

## Growth factor

In [6]:
import pandas as pd
np.set_printoptions(suppress=True)
print("n \t m \t d \t rep \t steps \t time \t\t alpha \t IsAut_Sp \t IsAut_Re \t steps_a \t time_a \t alpha_a \t IsAut_Sp_a \t IsAut_Re_a")
df = pd.DataFrame(columns=['n', 'm', 'd', 'rep', 'steps', 'time', 'alpha', 'x', 'IsAut_Sp', 'IsAut_Re', 'steps', 'time_a', 'alpha_a', 'x_a', '|a|', 'a', 'IsAut_Sp_a', 'IsAut_Re_a'])
row=0
NN=[10, 20, 30, 40, 50]
MM=[10, 20, 30, 40, 50]
for n in NN:
    for m in MM:
        D=[3,4,5]
        for d in D:
            for rep in [1,2,3,4,5]:
                input_matrix= np.loadtxt("instances/left_n%dm%dd%d_%d.txt"%(n,m,d,rep))
                output_matrix= np.loadtxt("instances/right_n%dm%dd%d_%d.txt"%(n,m,d,rep))
                #XX = code.growthRateGraph(output_matrix, input_matrix, 10000)
                XX = code.growthRateGraph(output_matrix, input_matrix, 10000)
                xsol = XX[0]
                alphasol =XX[1]
                step = XX[2]
                time=XX[4]
                XX_a = code.growthRateGraph_a(output_matrix, input_matrix, 10000)
                xsol_a = XX_a[0]
                alphasol_a =XX_a[1]
                step_a = XX_a[2]
                time_a=XX_a[4]
                asol=XX[5]
                # print("steps: ", step)
                # print("x: ", xsol)
                # print("alpha: ", alphasol)
                # print("time: ", time)
                isA_sp, isA_re=IsAutonomous(input_matrix, output_matrix, range(n), range(m))
                isA_sp_a, isA_re_a=IsAutonomous(input_matrix, output_matrix, asol, range(m))
                print("%d \t %d \t %d \t %d \t %d \t %.4f \t %.4f"%(n,m,d,rep,step,time,alphasol), "\t", len(asol), asol, isA_sp, isA_re, sep="\t")
                df.loc[row] = [n,m,d,rep,step,time,alphasol,xsol, step_a,time_a,alphasol_a,xsol_a,len(asol), asol, isA_sp_a, isA_re_a]
                row+=1
df.to_csv('GrowthRate_Experiments.csv', index=False)  

n 	 m 	 d 	 rep 	 steps 	 time 		 alpha 	 IsAut_Sp 	 IsAut_Re 	 steps_a 	 time_a 		 alpha_a 	 IsAut_Sp_a 	 IsAut_Re
10 	 10 	 3 	 1 	 26 	 0.5137 	 0.9611			10	[0 1 2 3 4 5 6 7 8 9]	True	True
10 	 10 	 3 	 2 	 24 	 0.4333 	 0.0654			8	[0 1 2 3 4 5 6 7]	True	True
10 	 10 	 3 	 3 	 17 	 0.2643 	 0.8187			8	[0 2 4 5 6 7 8 9]	True	True
10 	 10 	 3 	 4 	 32 	 0.5097 	 0.4282			9	[0 1 2 3 4 5 7 8 9]	True	True
10 	 10 	 3 	 5 	 0 	 0.0064 	 0.1691			9	[0 1 2 3 5 6 7 8 9]	False	True
10 	 10 	 4 	 1 	 0 	 0.0052 	 0.0000			9	[0 1 2 3 4 5 6 8 9]	False	False


/Users/vblanco/Library/Mobile Documents/com~apple~CloudDocs/projects/dynamic_autocatalysis/codeModel.py:359: RuntimeWarning: divide by zero encountered in scalar divide
  alpha_t = np.min([sum(output_matrix[s, r] * x_t[r]


10 	 10 	 4 	 2 	 28 	 0.3770 	 0.7915			10	[0 1 2 3 4 5 6 7 8 9]	True	True
10 	 10 	 4 	 3 	 0 	 0.0059 	 0.1784			8	[0 1 2 4 6 7 8 9]	False	False
10 	 10 	 4 	 4 	 0 	 0.0054 	 0.0000			8	[0 1 2 3 6 7 8 9]	False	False
10 	 10 	 4 	 5 	 0 	 0.0055 	 0.0192			9	[0 2 3 4 5 6 7 8 9]	False	False
10 	 10 	 5 	 1 	 0 	 0.0048 	 0.0000			8	[0 1 3 4 5 6 8 9]	False	False
10 	 10 	 5 	 2 	 0 	 0.0054 	 0.0000			8	[0 1 2 4 6 7 8 9]	False	False
10 	 10 	 5 	 3 	 0 	 0.0058 	 0.4022			9	[0 1 2 3 4 5 6 7 8]	False	False
10 	 10 	 5 	 4 	 6 	 0.0827 	 0.0001			8	[1 2 3 4 5 6 7 8]	True	False
10 	 10 	 5 	 5 	 0 	 0.0049 	 0.0000			8	[0 1 3 4 6 7 8 9]	False	False


/Users/vblanco/Library/Mobile Documents/com~apple~CloudDocs/projects/dynamic_autocatalysis/codeModel.py:359: RuntimeWarning: divide by zero encountered in scalar divide
  alpha_t = np.min([sum(output_matrix[s, r] * x_t[r]
/Users/vblanco/Library/Mobile Documents/com~apple~CloudDocs/projects/dynamic_autocatalysis/codeModel.py:359: RuntimeWarning: divide by zero encountered in scalar divide
  alpha_t = np.min([sum(output_matrix[s, r] * x_t[r]
/Users/vblanco/Library/Mobile Documents/com~apple~CloudDocs/projects/dynamic_autocatalysis/codeModel.py:359: RuntimeWarning: divide by zero encountered in scalar divide
  alpha_t = np.min([sum(output_matrix[s, r] * x_t[r]
/Users/vblanco/Library/Mobile Documents/com~apple~CloudDocs/projects/dynamic_autocatalysis/codeModel.py:359: RuntimeWarning: divide by zero encountered in scalar divide
  alpha_t = np.min([sum(output_matrix[s, r] * x_t[r]
/Users/vblanco/Library/Mobile Documents/com~apple~CloudDocs/projects/dynamic_autocatalysis/codeModel.py:359: Run


Interrupt request received
10 	 20 	 3 	 1 	 24 	 0.5527 	 -1.0000			0	[]	True	False
10 	 20 	 3 	 2 	 35 	 1.0761 	 1.6019			9	[0 1 2 4 5 6 7 8 9]	True	True
10 	 20 	 3 	 3 	 50 	 1.0982 	 2.3082			10	[0 1 2 3 4 5 6 7 8 9]	True	True
10 	 20 	 3 	 4 	 71 	 2.1728 	 1.8812			10	[0 1 2 3 4 5 6 7 8 9]	True	True
10 	 20 	 3 	 5 	 112 	 3.1293 	 1.8724			10	[0 1 2 3 4 5 6 7 8 9]	True	False


In [14]:
import pandas as pd
SM0 = pd.DataFrame(SM)
SM0.columns = ['']*SM0.shape[1]
print(SM0.to_string(index=False))

I0 = pd.DataFrame(input_matrix_sol)
I0.columns = ['']*I0.shape[1]
print(I0.to_string(index=False))

O0 = pd.DataFrame(output_matrix_sol)
O0.columns = ['']*O0.shape[1]
print(O0.to_string(index=False))

SMsol0 = pd.DataFrame(SMsol)
SMsol0.columns = ['']*SMsol0.shape[1]
print(SMsol0.to_string(index=False))

                                                                                                                       
-13.0  13.0 -10.0   3.0 -15.0  15.0 -10.0  -9.0  15.0   0.0   0.0 -12.0   0.0  17.0  10.0 -12.0  12.0   8.0  -4.0  -3.0
  0.0   2.0 -12.0   0.0   4.0 -18.0   0.0 -14.0  10.0  -5.0  16.0  14.0  -4.0  -5.0 -18.0   0.0  18.0 -12.0 -12.0   0.0
  0.0  10.0 -13.0  -2.0   6.0   0.0   0.0   0.0   0.0   0.0  -3.0 -19.0  10.0  -4.0 -15.0   3.0   0.0   0.0  -5.0   0.0
 -2.0  -4.0   0.0   0.0  11.0   0.0   0.0  -1.0   0.0  11.0  10.0   4.0 -14.0   0.0  -8.0   8.0   0.0  10.0 -13.0   4.0
  8.0  -1.0   6.0   3.0   0.0   9.0 -13.0  18.0 -13.0  11.0   9.0   0.0  -8.0   0.0   0.0   7.0  -7.0 -17.0   0.0  -7.0
-12.0  -5.0 -17.0   5.0  19.0   0.0 -17.0   0.0   0.0   0.0   0.0 -17.0 -17.0 -11.0  10.0 -11.0   7.0   0.0   0.0   0.0
-16.0  -9.0  -6.0   0.0   0.0   2.0  -7.0  -1.0  -5.0  -1.0   0.0   0.0  19.0   6.0  -5.0   0.0  19.0   8.0   0.0   6.0
  0.0   0.0  16.0 -19.0  19.0   0.0 -11.

In [15]:
alpha_hand =  np.min([sum(output_matrix[s, r] * xsol[r] 
                         for r in zsol)
                     /
                     sum(input_matrix[s, r] * xsol[r] 
                         for r in zsol) 
                     for s in asol if sum(input_matrix[s, r] * xsol[r] 
                         for r in zsol)>0.0001])
alpha_hand

15.684386344640638